# Histogram fits with `pyhf`

Often we don't have a clear way to parametrize our fit templates, so we need to resort to MC simulations and use histograms as templates that we fit to data in the same bins.

We are going to use the [`pyhf`](https://github.com/scikit-hep/pyhf) package for these fits. The documentation can be found at https://pyhf.readthedocs.io/.

It can be installed with pip, e.g.

`pip install --user pyhf`

In addition we are using the `mplhep` package in this notebook to make histogram plotting more convenient and `iminuit` to extract uncertainties on fit parameters.

`pip install --user mplhep iminuit`

In [ ]:
import pyhf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep

Let's create 2 artificial histograms with 10 bins (having 11 bin boundaries). You could imagine these as two different background processes for which we have MC simulations on which we ran some event selection and created histograms for. For now, let's assume that the shape of these distributions comes out correctly and we only need to fit the normalization (for both templates independently) to data.

In [ ]:
bins = np.arange(11)

In [ ]:
hist1 = np.array([1.5, 3., 6., 7.5, 6.3, 6.6, 6., 4.5, 3. , 1.5])
hist2 = np.array([3. , 6., 9., 12., 15., 9. , 6., 3. , 0.3, 0.15])

In [ ]:
hep.histplot([hist1, hist2], bins)

We want to stack them since we think the sum of both should give us the expected data yield

In [ ]:
hep.histplot([hist1, hist2], bins, stack=True, histtype="fill")

Now, let's assume we observed the following data counts in each bin:

In [ ]:
data = np.array([ 4, 17, 26, 23, 34, 23, 21,  7,  8,  4])

In [ ]:
hep.histplot([hist1, hist2], bins, stack=True, histtype="fill")
hep.histplot(data, bins, histtype="errorbar", color="black")

Oftentimes, one plots errorbars, indicating 1 $\sigma$ confidence intervals on poisson distributed event counts to have some visualization on the expected spread.

In [ ]:
hep.histplot([hist1, hist2], bins, stack=True, histtype="fill")
hep.histplot(data, bins, histtype="errorbar", color="black", w2=data)

## One template fits it all

`pyhf` does fits using the Maximum-Likelihood method and uses the HistFactory ([CERN-OPEN-2012-016](https://cds.cern.ch/record/1456844)) template. In the simplemost case the pdf (probability density function) is just a product of poisson counts in each bin:

$$p(n|\lambda) = \prod_{\mathrm{bin}\, b} \mathrm{Pois}(n_b | \lambda_b)$$

where $\mathrm{Pois}(n_b | \lambda_b)$ is the Poisson distribution for $\lambda_b$ expected and $n_b$ observed counts. In our case $\lambda_b$ would be given by

$$\lambda_b = \mu_1 b_1 + \mu_2 b_2$$

where $b_1$ and $b_2$ are the expected counts from our 2 histograms and $\mu_1$ and $\mu_2$ are the normalization factors we want to fit. This pdf will define the Likelihood function that is later maximized to give the best fitting parameter values.

The general template is more complicated, allowing for constraint terms and separation into arbitrary channels - we will come back to that later.

Models in `pyhf` are defined with a json-like specification. In our case we can define the model with the following:

In [ ]:
samples = [
    {
        "name": "sample1",
        "data": list(hist1),
        "modifiers": [
            {"name": "mu1", "type": "normfactor", "data" : None}
        ],
    },
    {
        "name": "sample2",
        "data": list(hist2),
        "modifiers": [
            {"name": "mu2", "type": "normfactor", "data" : None}
        ],
    },
]
spec = {"channels" : [{"name" : "singlechannel", "samples" : samples}]}

In [ ]:
# info: the `poi_name=None` is nescessary here since we don't want to do a hypothesis test
model = pyhf.Model(spec, poi_name=None)

We will now run a *maximum likelihood fit* that gives us the parameters that maximize the likelihood, the *maximum likelihood estimates* (mle).

In [ ]:
mu1, mu2 = pyhf.infer.mle.fit(data, model)

In [ ]:
mu1, mu2

We did not have to specify initial parameter values or bounds. For normalization factors the initial parameters are by default `1` and the bounds (fit range) is `[0, 10]`:

In [ ]:
model.config.suggested_init()

In [ ]:
model.config.suggested_bounds()

Let's look at the fitted templates, together with the data:

In [ ]:
#hep.histplot([mu1 * hist1, mu2 * hist2], bins, stack=True, histtype="fill")
hep.histplot([mu1 * hist1, mu2 * hist2], bins, stack=True, histtype="fill")
hep.histplot(data, bins, histtype="errorbar", color="black", w2=data)

<div class="alert alert-block alert-success">
    <b>Exercise:</b> Repeat the fit, but use the same normalization factor for both samples. <i>Hint:</i> parameters in the pyhf specification are just identified by their name and the same name can occur in multiple places in the spec.
</div>

## Uncertainties on fit parameters and the "post-fit" plot

Often, we are also interested in the uncertainties and correlations between fit parameters. We can use `iminuit` as a fitting backend for `pyhf` to extract them:

In [ ]:
pyhf.set_backend('numpy', 'minuit')

In [ ]:
parameters, correlations = pyhf.infer.mle.fit(data, model, return_uncertainties=True, return_correlations=True)

In [ ]:
parameters

In [ ]:
correlations

<div class="alert alert-block alert-success">
    <b>Questions</b>: Why are the fit parameters correlated? Why is the correlation coefficient negative?
</div>

We can visualize the impact of these uncertainties on our fit templates using [linear error propagation](https://en.wikipedia.org/wiki/Propagation_of_uncertainty).

In this simple case, let's calculate this manually:

$$\sigma_{\lambda_b}^2 = \left(\frac{\partial \lambda_b}{\partial \mu_1}\sigma_{\mu_1}\right)^2 + \left(\frac{\partial \lambda_b}{\partial \mu_2}\sigma_{\mu_2}\right)^2 + 2 \frac{\partial \lambda_b}{\partial \mu_1}\frac{\partial \lambda_b}{\partial \mu_2}\sigma_{\mu_1}\sigma_{\mu_2}\rho_{12} = \left(b_1\sigma_{\mu_1}\right)^2 + \left(b_2\sigma_{\mu_2}\right)^2 + 2 b_1 b_2 \sigma_{b_1}\sigma_{b_2}\rho_{12}$$

In [ ]:
sigma1, sigma2 = parameters[:, 1]
sigma1, sigma2

In [ ]:
hist_error = np.sqrt(
    (hist1 * sigma1) ** 2 + (hist2 * sigma2) ** 2 + 2 * hist1 * hist2 * sigma1 * sigma2 * correlations[0][1]
)
hist_error

For more generic templates/functions you can do that automatically. Either use the [`uncertainties`](https://pythonhosted.org/uncertainties/) package (for functions described by simple formulas) or calculate it numerically by varying each parameter up and down and using half of the resulting interval as a replacement for $\frac{\partial f}{\partial x_i}\sigma_{x_i}$.

A generic function for using this method with `pyhf` models is currently provided by the [`cabinetry`](https://github.com/alexander-held/cabinetry) package from Alexander Held with the function [`cabinetry.model_utils.calculate_stdev`](https://github.com/alexander-held/cabinetry/blob/c8668005e899556675b5e646e127908849bfe597/src/cabinetry/model_utils.py#L176)

In [ ]:
def errorband(bins, hist, hist_error, ax=None):
    ax = ax or plt.gca()
    n = hist
    s = hist_error

    def extend(x):
        return np.append(x, x[-1])

    ax.fill_between(
        bins,
        extend(n - s),
        extend(n + s),
        step="post",
        color="black",
        alpha=0.3,
        linewidth=0
    )

In [ ]:
hep.histplot([mu1 * hist1, mu2 * hist2], bins, stack=True, histtype="fill")
hep.histplot(data, bins, histtype="errorbar", color="black", w2=data)
errorband(bins, mu1 * hist1 + mu2 * hist2, hist_error)

This errorband is often referred to as "post-fit" uncertainties.

<div class="alert alert-block alert-info">
    <b>Note:</b> Since the same normalization factor is used across all bins, the uncertainty on the template depends on all data points. Therefore the interval is smaller than the expected fluctuation in each individual bin.
</div>

<div class="alert alert-block alert-success">
    <b>Question:</b> What would happen if you fit an independent template per histogram bin? What would the uncertainty look like?
</div>

<div class="alert alert-block alert-success">
    <b>Exercise [hard]:</b> Try it out! Hint: One way to do this is by defining a sample for each bin (with an individual normalization factor for each bin) and setting all other bins to 0.
</div>

# Advanced:  Uncertainties on the histogram templates